In [ ]:
import numpy as np
import os
import cv2
import torch
from torch.utils.data import Dataset
import csv
from torch.utils.data import DataLoader
from network_files import my_utility as mu
from utility_face_detection import face_detection, TrainDataset


In [ ]:
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def max_area(faces):
    max_area = 0
    for f in faces:
        (x1, y1, w1, h1) = f
        area = w1 * h1
        if(area > max_area):
            max_area = area
            (x, y, w, h) = f

    return (x, y, w, h)

In [ ]:
# Stesso codice di FaceDetection ma con altre cartelle
def face_detection_test_images(test_images):
    for i in range(1,25):
        
        write_path = test_images + "face_test_images\\" + str(i).rjust(2,'0')
        if not os.path.exists(write_path):
            os.makedirs(write_path)
        
        read_path = test_images + str(i).rjust(2,'0')
        
        print(f"Cartella Numero {str(i)}")
        for file_name in os.listdir(read_path):
            if not file_name.startswith('.'):
                image_path = read_path + '\\' + file_name
                img = cv2.imread(image_path)
                gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                faces = face_classifier.detectMultiScale(
                    gray_image, scaleFactor=1.05, minNeighbors=12, minSize=(90, 90), maxSize=(150, 150)
                )  
                if len(faces) > 1:
                    x, y, w, h = max_area(faces)
                else:
                    x, y, w, h = faces[0]
                
                img_crop = gray_image[y:y+h, x:x+w]

                cv2.imwrite(write_path + '\\' + file_name, img_crop)
 

In [1]:
# Stesso codice di codifica del train ma senza i labels
def codify_test_images(test_images):
    model = mu.SiameseNeuralNetwork()
    model_path = torch.load(f="model_all_images")
    model.load_state_dict(model_path)
    model.eval()
    with open("test_template.csv", "w", newline="") as template:
        writer = csv.writer(template, delimiter=";")
        for i in range(1,25):
            train_curr_images = test_images + "\\" + str(i).rjust(2,'0')
            images = TrainDataset(img_dir=train_curr_images)
            img_dataloader = DataLoader(images, batch_size=1, drop_last=True)
            for image in img_dataloader:
                to_write = list(model(image).cpu().detach().numpy().flatten())
                writer.writerow([to_write])

In [ ]:
test_directory = 'C:\\Users\\antho\\Desktop\\faceDetection\\dataset_splitted\\test\\'

face_detection_test_images(test_directory)
codify_test_images(test_directory + "face_test_images\\")